<a href="https://colab.research.google.com/github/latkow/materials/blob/main/szkolenie_20210705.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sekcja: instalacja pakietów
Zainstalujemy pakiet `survey` oraz `vcd` (Visualisation of Categorical Data)

Załadujmy pakiety na potrzeby dzisiejszej pracy.

In [2]:
install.packages('survey')

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘minqa’, ‘numDeriv’, ‘mitools’




In [3]:
install.packages("vcd")

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘zoo’, ‘lmtest’




In [4]:
library(survey) # wykorzystamy do post-stratyfikacji, kalibracij
library(vcd) # do wsp. korelacji dla cech jakościowych
library(tidyverse) ## przetwarzanie i wizualizacja danych

Loading required package: grid

Loading required package: Matrix

Loading required package: survival


Attaching package: ‘survey’


The following object is masked from ‘package:graphics’:

    dotchart


Warning message in system("timedatectl", intern = TRUE):
“running command 'timedatectl' had status 1”
── Attaching packages ─────────────────────────────────────── tidyverse 1.3.1 ──

✔ ggplot2 3.3.3     ✔ purrr   0.3.4
✔ tibble  3.1.2     ✔ dplyr   1.0.6
✔ tidyr   1.1.3     ✔ stringr 1.4.0
✔ readr   1.4.0     ✔ forcats 0.5.1

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ tidyr::expand() masks Matrix::expand()
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
✖ tidyr::pack()   masks Matrix::pack()
✖ tidyr::unpack() masks Matrix::unpack()



Wczytajmy dane
bkl - badanie kapitału ludzkiego

zmiennymi celu ($Y$) są dla nas kolumny zaczynające się od `komp_` czyli różne kompetencje. Natomiast zmiennymi objaśniającymi/pomocniczymi ($X$) to zawód, województwo i sekcja pkd.

In [5]:
bkl <- readRDS('bkl.rds')
head(bkl)

,id,zawod1,zawod2,zawod6,woj,pkd,komp_techniczne,komp_matematyczne,komp_kulturalne,komp_komputerowe,komp_kognitywne,komp_kierownicze,komp_interpersonalne,komp_indywidualne,komp_fizyczne,komp_dyspozycyjne,komp_biurowe
,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
42425,626307,5,51,514202,04,M,0,0,1,0,0,0,0,1,0,0,0
42426,626305,5,51,514204,12,M,0,0,0,0,0,0,0,0,0,0,0
42427,617154,7,71,713104,14,C,0,0,0,0,0,0,0,0,0,0,0
42428,617155,7,71,713100,14,C,0,0,0,0,0,0,0,0,0,0,0
42429,632044,3,33,332203,24,C,0,0,0,0,0,0,1,0,1,0,0
42430,613019,3,33,332101,14,K,0,0,0,0,0,0,1,1,0,0,0


In [9]:
dim(bkl)

[1] 12904    17

Baniem, które chcemy wykorzystać do korektu będów nielosowych w BKL będzie badanie Popyt na Pracę GUS.

In [8]:
popyt <- readRDS("popyt.rds")
head(popyt)

,id_popyt,woj,waga,sektor,klasa,pkd,zawod1,wakaty
,<chr>,<chr>,<dbl>,<chr>,<chr>,<chr>,<fct>,<dbl>
1,1da98b1d90d55a45173fbf52273388d023f88bda,30,18,2,M,J,3,1
2,ba9c01a6d5941fb21c6d85d5abee6714d5636ee4,10,1,2,D,C,8,10
3,772ab1f561648f4125d0432456225d74a083b5bb,22,33,2,M,I,5,5
4,af0b53b2f75d7edf652af5e3f65553641f1f587a,14,1,2,D,Q,4,3
5,ab7c3795293f57e4d68f8638878f0a9278e3936a,30,1,1,D,C,7,6
6,fd8b0d69ae13f2d1baee41f0f69a7da82be04137,10,1,1,S,G,2,1


In [10]:
dim(popyt)

[1] 10422     8

In [11]:
sum(popyt$waga) # sumowanie do liczby podmiotów ze zgłoszonymi wolnymi miejscami pracy

[1] 66021

In [12]:
c("Liczba podmiotów" = sum(popyt$waga), "Liczba zgłoszonych wakatów" = sum(popyt$waga* popyt$wakaty))

Liczba podmiotów Liczba zgłoszonych wakatów 
                     66021                     155373

## Sprawdzamy zmienne X i Y

Sprawdźmy jak dobrze skorelowane są X (czyli pk, zawod1, woj) ze zmiennymi celu Y (kompetencje komputerowe i interpersonalne)

In [15]:
tab1a <- xtabs(~ komp_komputerowe + pkd, data=bkl)
tab1b <- xtabs(~ komp_komputerowe + zawod1, data=bkl)
tab2a <- xtabs(~ komp_interpersonalne + pkd, data=bkl)
tab2b <- xtabs(~ komp_interpersonalne + zawod1, data=bkl)

## w pakiecie vsd mamy funkcję assocstats, która liczby wsp. V-Cramer'a
assocstats(tab1a) ## 0.304 
assocstats(tab1b) ## 0.288
assocstats(tab2a) ## 0.274
assocstats(tab2b) ## 0.352

                    X^2 df P(> X^2)
Likelihood Ratio 1173.2 16        0
Pearson          1189.5 16        0

Phi-Coefficient   : NA 
Contingency Coeff.: 0.291 
Cramer's V        : 0.304 

                    X^2 df P(> X^2)
Likelihood Ratio 1185.9  7        0
Pearson          1073.9  7        0

Phi-Coefficient   : NA 
Contingency Coeff.: 0.277 
Cramer's V        : 0.288 

                     X^2 df P(> X^2)
Likelihood Ratio 1059.85 16        0
Pearson           969.11 16        0

Phi-Coefficient   : NA 
Contingency Coeff.: 0.264 
Cramer's V        : 0.274 

                    X^2 df P(> X^2)
Likelihood Ratio 1659.5  7        0
Pearson          1599.2  7        0

Phi-Coefficient   : NA 
Contingency Coeff.: 0.332 
Cramer's V        : 0.352 

## Post-stratyfikacja
Do post-stratyfikacji potrzebujemy wartości globalnych dotyczących liczby wakatów. Na potrzeby tego przykłądu wykorzystamy dwie zmienne: sekcję PKD (`pkd`) oraz kod zawodu (`zawod1`).

Aby przypisać wagi do zbioru `bkl` wykorzystamy funkcję z pakiety `tidyverse`, a dokładnie funkcję `count` oraz `left_join`.

Post-stratyfikację dokonamy w dwóch krokach:
1. z wykorzystaniem funkcji `count` wyznaczymy liczbę wakatów wg sekcji PKD oraz zawodów,
2. dołączymy tę informację do zbioru `bkl` i wykorzystamy do estymacji.

In [17]:
wartosci_globalne <- popyt %>% count(pkd, zawod1 = as.character(zawod1), wt = waga * wakaty, name ="total")
wartosci_globalne

pkd,zawod1,total
<chr>,<chr>,<dbl>
A,2,22
A,3,24
A,4,8
A,5,37
A,7,85
A,8,54
A,9,258
B,1,10
B,2,66


drugi krok -- przygotowujemy dane z bkl

In [18]:
bkl_komp <- bkl %>% count(pkd, zawod1, komp_komputerowe)
bkl_inter <- bkl %>% count(pkd, zawod1, komp_interpersonalne)
head(bkl_komp)

,pkd,zawod1,komp_komputerowe,n
,<chr>,<chr>,<int>,<int>
1,C,1,0,244
2,C,1,1,86
3,C,2,0,818
4,C,2,1,620
5,C,3,0,812
6,C,3,1,349


dołączamy informacje o liczbie wakatów w populacji (zbiór `wartosci_globalne`)

In [25]:
bkl_komp_total <- bkl_komp %>% left_join(wartosci_globalne)
bkl_komp_total

Joining, by = c("pkd", "zawod1")



pkd,zawod1,komp_komputerowe,n,total
<chr>,<chr>,<int>,<int>,<dbl>
C,1,0,244,523
C,1,1,86,523
C,2,0,818,2806
C,2,1,620,2806
C,3,0,812,1737
C,3,1,349,1737
C,4,0,91,914
C,4,1,73,914
C,5,0,449,368


In [28]:
bkl_komp_total <- bkl_komp %>% left_join(wartosci_globalne) %>% mutate(waga = total/n)
bkl_inter_total <- bkl_inter %>% left_join(wartosci_globalne) %>% mutate(waga = total/n)
head(bkl_inter_total)

Joining, by = c("pkd", "zawod1")

Joining, by = c("pkd", "zawod1")



,pkd,zawod1,komp_interpersonalne,n,total,waga
,<chr>,<chr>,<int>,<int>,<dbl>,<dbl>
1,C,1,0,143,523,3.657343
2,C,1,1,187,523,2.796791
3,C,2,0,478,2806,5.870293
4,C,2,1,960,2806,2.922917
5,C,3,0,383,1737,4.535248
6,C,3,1,778,1737,2.232648


In [30]:
## estymator post-stratyfikacyjny
weighted.mean(bkl_inter_total$komp_interpersonalne, bkl_inter_total$waga)
## estymator naiwny
weighted.mean(bkl_inter_total$komp_interpersonalne, bkl_inter_total$n)
## estymator naiwny wskazuje na nieznaczne przeszacowanie

[1] 0.5735054

[1] 0.6106634

Wykonujemy post-stratyfikację z pakietem `survey`

In [36]:
## dodawanie pseudo wagi
liczba_wakatow <- sum(popyt$waga * popyt$wakaty)
liczba_wakatow

liczba_ofert_bkl <- nrow(bkl)
liczba_ofert_bkl

bkl <- bkl %>% mutate(pseudo_waga = liczba_wakatow / liczba_ofert_bkl)
head(bkl)

[1] 155373

[1] 12904

,id,zawod1,zawod2,zawod6,woj,pkd,komp_techniczne,komp_matematyczne,komp_kulturalne,komp_komputerowe,komp_kognitywne,komp_kierownicze,komp_interpersonalne,komp_indywidualne,komp_fizyczne,komp_dyspozycyjne,komp_biurowe,pseudo_waga
,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<dbl>
42425,626307,5,51,514202,04,M,0,0,1,0,0,0,0,1,0,0,0,12.04069
42426,626305,5,51,514204,12,M,0,0,0,0,0,0,0,0,0,0,0,12.04069
42427,617154,7,71,713104,14,C,0,0,0,0,0,0,0,0,0,0,0,12.04069
42428,617155,7,71,713100,14,C,0,0,0,0,0,0,0,0,0,0,0,12.04069
42429,632044,3,33,332203,24,C,0,0,0,0,0,0,1,0,1,0,0,12.04069
42430,613019,3,33,332101,14,K,0,0,0,0,0,0,1,1,0,0,0,12.04069


Deklarujemy obiekt typy `svydesign` w pakiecie `survey`

In [37]:
bkl_surv <- svydesign(id = ~1, weights = ~ pseudo_waga, data=bkl) ## id = ~1 ifnormuje funkcję, że każdy rekord jest losowany niezależnie (opis schematu losowania)
bkl_surv

Independent Sampling design (with replacement)
svydesign(id = ~1, weights = ~pseudo_waga, data = bkl)

Chcemy oszacować odsetek ofert pracy, w których wystęują kompetencje komputerowe czy interpersonalne

In [39]:
svymean(~komp_komputerowe + komp_interpersonalne, design = bkl_surv)

                        mean     SE
komp_komputerowe     0.34323 0.0042
komp_interpersonalne 0.61066 0.0043

In [41]:
summary(bkl)

       id            zawod1             zawod2             zawod6         
 Min.   :601002   Length:12904       Length:12904       Length:12904      
 1st Qu.:610334   Class :character   Class :character   Class :character  
 Median :623138   Mode  :character   Mode  :character   Mode  :character  
 Mean   :622935                                                           
 3rd Qu.:634203                                                           
 Max.   :647117                                                           
     woj                pkd            komp_techniczne  komp_matematyczne
 Length:12904       Length:12904       Min.   :0.0000   Min.   :0.00000  
 Class :character   Class :character   1st Qu.:0.0000   1st Qu.:0.00000  
 Mode  :character   Mode  :character   Median :0.0000   Median :0.00000  
                                       Mean   :0.0444   Mean   :0.00341  
                                       3rd Qu.:0.0000   3rd Qu.:0.00000  
                               

Dokonujemy post-stratyfikacji

In [43]:
## musze zapewnić, że dane będą tego samego typu
head(popyt)
popyt <- popyt %>% mutate(zawod1 = as.character(zawod1))
head(popyt)

,id_popyt,woj,waga,sektor,klasa,pkd,zawod1,wakaty
,<chr>,<chr>,<dbl>,<chr>,<chr>,<chr>,<fct>,<dbl>
1,1da98b1d90d55a45173fbf52273388d023f88bda,30,18,2,M,J,3,1
2,ba9c01a6d5941fb21c6d85d5abee6714d5636ee4,10,1,2,D,C,8,10
3,772ab1f561648f4125d0432456225d74a083b5bb,22,33,2,M,I,5,5
4,af0b53b2f75d7edf652af5e3f65553641f1f587a,14,1,2,D,Q,4,3
5,ab7c3795293f57e4d68f8638878f0a9278e3936a,30,1,1,D,C,7,6
6,fd8b0d69ae13f2d1baee41f0f69a7da82be04137,10,1,1,S,G,2,1


,id_popyt,woj,waga,sektor,klasa,pkd,zawod1,wakaty
,<chr>,<chr>,<dbl>,<chr>,<chr>,<chr>,<chr>,<dbl>
1,1da98b1d90d55a45173fbf52273388d023f88bda,30,18,2,M,J,3,1
2,ba9c01a6d5941fb21c6d85d5abee6714d5636ee4,10,1,2,D,C,8,10
3,772ab1f561648f4125d0432456225d74a083b5bb,22,33,2,M,I,5,5
4,af0b53b2f75d7edf652af5e3f65553641f1f587a,14,1,2,D,Q,4,3
5,ab7c3795293f57e4d68f8638878f0a9278e3936a,30,1,1,D,C,7,6
6,fd8b0d69ae13f2d1baee41f0f69a7da82be04137,10,1,1,S,G,2,1


Tworzymy dane wejściowe do post-stratyfikacji

In [44]:
tab_pop_zawod1 <- xtabs(waga*wakaty ~ zawod1, data=popyt) # suma ważona po jednej zmiennej (zawod1)
tab_pop_zawod1 # tabela z wartościami globalnymi
# dla zawodu 1 było 3178 ofert pracy w badaniu popytu na pracę

zawod1
    1     2     3     4     5     7     8     9 
 3178 21932  8825 14839 17309 49283 25872 14135 

In [47]:
bkl_surv_ps <- postStratify(design = bkl_surv, strata = ~zawod1, population = tab_pop_zawod1)
bkl_surv_ps

Independent Sampling design (with replacement)
postStratify(design = bkl_surv, strata = ~zawod1, population = tab_pop_zawod1)

In [49]:
svymean(~komp_komputerowe + komp_interpersonalne, design = bkl_surv) # wynik przed stratyfikacją
svymean(~komp_komputerowe + komp_interpersonalne, design = bkl_surv_ps) # wynik po stratyfikacji
## 0.34 --> 0.20 po uwzględnieniu różnić w strukturze zawodów w badaniu bkl i badaniu Popyt na pracę GUS

                        mean     SE
komp_komputerowe     0.34323 0.0042
komp_interpersonalne 0.61066 0.0043

                        mean     SE
komp_komputerowe     0.20432 0.0044
komp_interpersonalne 0.34320 0.0061

In [51]:
bkl$pseudo_waga_ps <- weights(bkl_surv_ps)
head(bkl)
sum(bkl$pseudo_waga_ps) # 155373

,id,zawod1,zawod2,zawod6,woj,pkd,komp_techniczne,komp_matematyczne,komp_kulturalne,komp_komputerowe,komp_kognitywne,komp_kierownicze,komp_interpersonalne,komp_indywidualne,komp_fizyczne,komp_dyspozycyjne,komp_biurowe,pseudo_waga,pseudo_waga_ps
,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<dbl>,<dbl>
42425,626307,5,51,514202,04,M,0,0,1,0,0,0,0,1,0,0,0,12.04069,5.188549
42426,626305,5,51,514204,12,M,0,0,0,0,0,0,0,0,0,0,0,12.04069,5.188549
42427,617154,7,71,713104,14,C,0,0,0,0,0,0,0,0,0,0,0,12.04069,66.869742
42428,617155,7,71,713100,14,C,0,0,0,0,0,0,0,0,0,0,0,12.04069,66.869742
42429,632044,3,33,332203,24,C,0,0,0,0,0,0,1,0,1,0,0,12.04069,3.121684
42430,613019,3,33,332101,14,K,0,0,0,0,0,0,1,1,0,0,0,12.04069,3.121684


[1] 155373

## Kalibracja

Do kalibracji użyjemy trzy zmienne: `pkd` , `zawod1` oraz `woj`

In [53]:
tab_pop_pkd <- xtabs(waga*wakaty ~ pkd, data=popyt)
tab_pop_woj <- xtabs(waga*wakaty ~ woj, data=popyt)
tab_pop_pkd
tab_pop_zawod1
tab_pop_woj

pkd
    A     B     C     D     E     F     G     H     I     J     K     L     M 
  488   255 36965   509  1128 23386 34849 12331  6751  6214  2584  1531  7300 
    N     O     P     Q     R     S 
 4828  5434  2013  5339   779  2689 

zawod1
    1     2     3     4     5     7     8     9 
 3178 21932  8825 14839 17309 49283 25872 14135 

woj
   02    04    06    08    10    12    14    16    18    20    22    24    26 
13725 13032  4358  2906  7957 18663 30555  2391  4888  1789 10573 16881  2073 
   28    30    32 
 3428 16805  5349 

Dokonujemy klibracji w wykorzystaniem funkcji `calibrate`

In [59]:
bkl_surv_cal <- calibrate(design = bkl_surv,
                formula = list(~zawod1, ~woj), 
                population = list(tab_pop_zawod1, tab_pop_woj))
bkl_surv_cal
bkl$pseudo_waga_cal <- weights(bkl_surv_cal)
head(bkl)

Independent Sampling design (with replacement)
calibrate(design = bkl_surv, formula = list(~zawod1, ~woj), population = list(tab_pop_zawod1, 
    tab_pop_woj))

,id,zawod1,zawod2,zawod6,woj,pkd,komp_techniczne,komp_matematyczne,komp_kulturalne,komp_komputerowe,komp_kognitywne,komp_kierownicze,komp_interpersonalne,komp_indywidualne,komp_fizyczne,komp_dyspozycyjne,komp_biurowe,pseudo_waga,pseudo_waga_ps,pseudo_waga_cal
,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<dbl>,<dbl>,<dbl>
42425,626307,5,51,514202,04,M,0,0,1,0,0,0,0,1,0,0,0,12.04069,5.188549,11.207997
42426,626305,5,51,514204,12,M,0,0,0,0,0,0,0,0,0,0,0,12.04069,5.188549,8.924961
42427,617154,7,71,713104,14,C,0,0,0,0,0,0,0,0,0,0,0,12.04069,66.869742,69.958450
42428,617155,7,71,713100,14,C,0,0,0,0,0,0,0,0,0,0,0,12.04069,66.869742,69.958450
42429,632044,3,33,332203,24,C,0,0,0,0,0,0,1,0,1,0,0,12.04069,3.121684,2.348824
42430,613019,3,33,332101,14,K,0,0,0,0,0,0,1,1,0,0,0,12.04069,3.121684,6.099183


Porównujemy wynikimi trzech estymatorów: + naiwnego, + postratyfikacyjnego i kalibracyjnego

In [60]:
svymean(~komp_komputerowe + komp_interpersonalne, design = bkl_surv) # wynik przed stratyfikacją
svymean(~komp_komputerowe + komp_interpersonalne, design = bkl_surv_ps)
svymean(~komp_komputerowe + komp_interpersonalne, design = bkl_surv_cal) # wychodzi na to, że wojewodztwo niewiele nam wnosi


                        mean     SE
komp_komputerowe     0.34323 0.0042
komp_interpersonalne 0.61066 0.0043

                        mean     SE
komp_komputerowe     0.20432 0.0044
komp_interpersonalne 0.34320 0.0061

                        mean     SE
komp_komputerowe     0.20764 0.0046
komp_interpersonalne 0.34169 0.0062

Podsumowanie:
1. aby skorygować wyniki z próby nielosowej należy poszukać jakiegoś zewnętrzengo (do próby nielosowej) źródła danych, np. spis, rejestry administracyjne, próby losowe;
2. trzeba sprawdzić jakie zmienne $X$ (demograficzna i inne charakterystyki) korelują z przynależnością do próby nielosowej (oznaczone jako R/delta) i zmiennej celu ($Y$) --> im lepsza (wyższa) korelacja tym wykorzystanie tych zmiennych powinno zmniejszyć obciążenie.
3. dobór zmiennych jest kluczowy --> w szczegółach będzie o tym jutro
4. trzeba ujednolicić zmienne (aby w obu zbiorach były takie same)
5. wybór metody: post-stratyfikacja czy kalibracja? Warto wybrać kalibrację z racji tego, że jest ona bardziej elastyczna choć wymaga oprogramowania
6. przeprowadzenie estymacji z wykorzystaniem wag, które uzyskamy z kroku 5

In [62]:
weighted.mean(bkl$komp_interpersonalne, bkl$pseudo_waga)
weighted.mean(bkl$komp_interpersonalne, bkl$pseudo_waga_cal)
weighted.mean(bkl$komp_interpersonalne, bkl$pseudo_waga_ps)

[1] 0.6106634

[1] 0.3416862

[1] 0.3432004

In [63]:
weighted.mean(bkl$komp_techniczne, bkl$pseudo_waga)
weighted.mean(bkl$komp_techniczne, bkl$pseudo_waga_cal)
weighted.mean(bkl$komp_techniczne, bkl$pseudo_waga_ps)

[1] 0.04440484

[1] 0.1042676

[1] 0.1042938